In [1]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 28.7 MB/s 


In [2]:
import os
import csv
import pdb
import math
import time 
import pickle
import pickle
import random
import warnings
import datetime 
import collections
import numpy as np 
import pandas as pd  
import xgboost as xgb
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pylab as plt 
from matplotlib import rcParams   
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import time

# Algo Import
import tensorflow as tf
from tensorflow import keras
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
data_path = "gdrive/My Drive/Major Project/"

In [ ]:
combined_tr = pd.read_csv(data_path+"/train_91.csv", index_col = 0)
print(combined_tr.shape)
combined_tr.head()

(902474, 3)


,src,dest,link
73132,35080,10701,0
404654,54809,75095,0
310475,67628,60404,0
301271,57253,462,0
332230,15167,47459,1


In [ ]:
combined_te = pd.read_csv(data_path+"/test_91.csv", index_col = 0)
print(combined_te.shape)
combined_te.head()

(106363, 3)


,src,dest,link
487942,32198,62377,0
103900,6056,11650,1
78661,2040,22467,1
71074,143,0,1
382438,1609,5035,1


In [ ]:
df_train = pd.read_csv(data_path+"pos_train_91.csv", index_col=0)
print(df_train.shape)
df_train.head()

(452474, 3)


,src,dest,link
496205,54500,16485,1
141147,3318,30547,1
483066,33514,4007,1
478351,28118,2641,1
505144,69985,61687,1


In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())

In [ ]:
degrees = nx.degree(train_graph)
degrees = dict(degrees)

features = {}
for key, val in degrees.items():
  features[key] = [val]

pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

In [ ]:
nx.set_node_attributes(train_graph, features, "features")

In [ ]:
G = StellarGraph.from_networkx(train_graph, node_features="features")
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 75879, Edges: 452474

 Node types:
  default: [75879]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [452474]
        Weights: all 1 (default)
        Features: none


In [ ]:
fullbatch_generator = FullBatchNodeGenerator(G, sparse=True)
gcn_model = GCN(layer_sizes=[64], activations=['relu'], generator=fullbatch_generator)

corrupted_generator = CorruptedGenerator(fullbatch_generator)
gen = corrupted_generator.flow(G.nodes())

Using GCN (local pooling) filters...


In [ ]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
batch_size=128
epochs=20
es = EarlyStopping(monitor="loss", min_delta=0, patience=4, restore_best_weights=True,verbose=1)

In [ ]:
history = model.fit(gen, epochs=epochs, verbose=1, callbacks=[es])
model_file = open(data_path+"gcn_model.h5", "wb")
pickle.dump(model, model_file)
model_file.close()

Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 7.6237
Epoch 2/20
1/1 [==============================] - 0s 98ms/step - loss: 5.4286
Epoch 3/20
1/1 [==============================] - 0s 99ms/step - loss: 3.2795
Epoch 4/20
1/1 [==============================] - 0s 105ms/step - loss: 1.2604
Epoch 5/20
1/1 [==============================] - 0s 95ms/step - loss: 0.9234
Epoch 6/20
1/1 [==============================] - 0s 102ms/step - loss: 1.4701
Epoch 7/20
1/1 [==============================] - 0s 93ms/step - loss: 1.9125
Epoch 8/20
1/1 [==============================] - 0s 96ms/step - loss: 2.2058
Epoch 9/20
1/1 [==============================] - 0s 105ms/step - loss: 2.3887
Epoch 9: early stopping
INFO:tensorflow:Assets written to: ram://a5ece139-f7ae-433a-b9d6-ceccae1dd1fd/assets


In [ ]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = Model(inputs=x_emb_in, outputs=x_out)

In [ ]:
train_gen = fullbatch_generator.flow(G.nodes())
emb = emb_model.predict(train_gen)

In [ ]:
emb_dict = {}
for i, node in enumerate(list(G.nodes())):
  emb_dict[node] = np.array(emb[i].reshape(1,64))

In [ ]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined_tr.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)
print(train_emb.shape)
train_y = combined_tr['link']
print(len(train_y))

0it [00:00, ?it/s]

(902474, 64)
902474


In [ ]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_te.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])
print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_te['link'] 
print(len(test_y))

0it [00:00, ?it/s]

<class 'list'>
(106363, 64)
106363


In [ ]:
pickle.dump((train_emb, train_y), open(data_path + 'epinions_gcn_train91.pkl', 'wb'))
pickle.dump((test_emb, test_y), open(data_path + 'epinions_gcn_test91.pkl', 'wb'))

In [5]:
train_emb, train_y = pickle.load(open(data_path + 'epinions_gcn_train91.pkl', 'rb'))
test_emb, test_y = pickle.load(open(data_path + 'epinions_gcn_test91.pkl', 'rb'))

In [ ]:
# train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [ ]:
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for XGB fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for XGB fit =  496.96788144111633
Time taken for train data precition =  3.529895782470703
Time taken for test data precition =  0.4226064682006836
Train Score 0.8742030208147439 test Score 0.8876991502307046


In [ ]:
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for RandomForest fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for RandomForest fit =  405.21961426734924
Time taken for train data precition =  6.968756914138794
Time taken for test data precition =  0.8528940677642822
Train Score 0.8770638457248143 test Score 0.8883932289979546


In [6]:
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
start = time.time()
clf.fit(train_emb, train_y)
print("Time taken for LGBM fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y,clf.predict(train_emb))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for LGBM fit =  14.42127537727356
Time taken for train data precition =  3.159679889678955
Time taken for test data precition =  0.38519883155822754
Train Score 0.8724944958448402 test Score 0.8882784051624799
